In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
pip install torch torchvision matplotlib

# **1. Import Libraries** 

In [ ]:
import os
import shutil
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


# **2. Load Dataset** 

In [ ]:
# Source and destination directories
src_dir = '/kaggle/input/satellite-image-classification/data'
dst_dir = '/kaggle/working/data'
os.makedirs(dst_dir, exist_ok=True)

for split in ['train', 'val']:
    for cls in os.listdir(src_dir):
        os.makedirs(os.path.join(dst_dir, split, cls), exist_ok=True)

# Split and copy
split_ratio = 0.8

for cls in os.listdir(src_dir):
    imgs = os.listdir(os.path.join(src_dir, cls))
    random.shuffle(imgs)

    split_point = int(len(imgs) * split_ratio)
    train_imgs = imgs[:split_point]
    val_imgs = imgs[split_point:]

    for img in train_imgs:
        shutil.copy(os.path.join(src_dir, cls, img), os.path.join(dst_dir, 'train', cls, img))

    for img in val_imgs:
        shutil.copy(os.path.join(src_dir, cls, img), os.path.join(dst_dir, 'val', cls, img))

print("Dataset split into train/val successfully.")


In [ ]:
train_dir = '/kaggle/working/data/train'
val_dir = '/kaggle/working/data/val'

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ])
}

datasets_ = {
    'train': datasets.ImageFolder(train_dir, transform=data_transforms['train']),
    'val': datasets.ImageFolder(val_dir, transform=data_transforms['val'])
}

dataloaders = {
    'train': DataLoader(datasets_['train'], batch_size=32, shuffle=True),
    'val': DataLoader(datasets_['val'], batch_size=32)
}

class_names = datasets_['train'].classes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# **3. Define Custom CNN**

In [ ]:
class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 112x112
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 56x56
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)   # 28x28
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 28 * 28, 256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


# **4. Define Train Function**

In [ ]:
def train_model(model, criterion, optimizer, epochs=10):
    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        for phase in ['train', 'val']:
            model.train() if phase == 'train' else model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels)

            epoch_loss = running_loss / len(datasets_[phase])
            epoch_acc = running_corrects.double() / len(datasets_[phase])
            print(f"{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")


# **5. Train Custom CNN**

In [ ]:
model_cnn = CustomCNN(num_classes=len(class_names)).to(device)
optimizer_cnn = optim.Adam(model_cnn.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

train_model(model_cnn, criterion, optimizer_cnn, epochs=10)

# **6. Transfer Learning (ResNet18)**

In [ ]:
model_resnet = models.resnet18(pretrained=True)

# Freeze all layers
for param in model_resnet.parameters():
    param.requires_grad = False

# Replace the final layer
model_resnet.fc = nn.Linear(model_resnet.fc.in_features, len(class_names))
model_resnet = model_resnet.to(device)

optimizer_resnet = optim.Adam(model_resnet.fc.parameters(), lr=0.001)
train_model(model_resnet, criterion, optimizer_resnet, epochs=10)


# **7. Evaluate Models**

In [ ]:
def evaluate_model(model, title="Model"):
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for inputs, labels in dataloaders['val']:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    print(f"\n📈 Classification Report for {title}:")
    print(classification_report(y_true, y_pred, target_names=class_names))

    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6,5))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_names,
                yticklabels=class_names, cmap="Blues")
    plt.title(f"Confusion Matrix - {title}")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()


# **8. Final Evaluation**

In [ ]:
evaluate_model(model_cnn, title="Custom CNN")
evaluate_model(model_resnet, title="ResNet18 (Transfer Learning)")
